In [11]:
import torch
import torchvision
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from utils.tools import dotdict
from driver.driver import ABC_Driver
from torch_geometric_temporal import METRLADatasetLoader
from other_model.other_model import make_default_model
import atd2022
torch.cuda.is_available()
torch.cuda.set_device(2)

In [ ]:
args = dotdict()
args.name = 'cifar100'

args.train_batch_size = 60
args.predict_batch_size = 100
args.train_epochs = 2
args.lr = 0.0001
args.criterion = 'CE'
args.scheduler = None

input_channel = 3
pixel_number = 32*32

args.layers=[
    ('linear', (input_channel*pixel_number, input_channel*2*pixel_number, (1,2,3), None, None, 'relu')),
    ('linear', (input_channel*2*pixel_number, 100, -1, None, None, None)),
    ('softmax', (1))
]

In [ ]:
driver = ABC_Driver(args, None, record_path=False)
driver.train()

In [12]:
model = torchvision.models.resnet18(weights="DEFAULT")

In [13]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  